# CNN architecture

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)  # MNIST has 1 input channel
        # 28 - 3 + 2 * 1 + 1= 28
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # 28 - 2/2 + 1= 14
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        # 14 - 3 + 2 * 1 + 1= 14
        # 14 - 2/2 + 1= 7

        # Set the flattened size after convolutions and pooling
        self.flattened_size_conv1 = 16 * 14 * 14  # Size after flattening conv1 and pooling
        self.flattened_size_conv2 = 32 * 7 * 7    # Size after flattening conv2 and pooling

        self.fc1 = nn.Linear(self.flattened_size_conv1 + self.flattened_size_conv2, 512)
        self.fc2 = nn.Linear(512, 10)  # 10 classes for MNIST

    def forward(self, x):
        x1 = self.pool(F.relu(self.conv1(x)))
        #print('x1',x1.shape)
        x2 = self.pool(F.relu(self.conv2(x1)))
        #print('x2',x2.shape)

        x1_flat = x1.view(x1.size(0), -1)
        #print('x1_flat',x1_flat.shape)
        x2_flat = x2.view(x2.size(0), -1)
        #print('x2_flat',x2_flat.shape)

        combined_flat = torch.cat((x1_flat, x2_flat), dim=1)
        #print('combined_flat',combined_flat.shape)

        x = F.relu(self.fc1(combined_flat))
        #print('x fc1',x.shape)
        x = self.fc2(x)
        #print('x fc2',x.shape)

        return x

# Dataset Loading

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=1000, shuffle=False)

model = CustomCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4547922.50it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 133175.27it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1259905.13it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3032075.25it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



# Training

In [3]:
# Training loop
for epoch in range(5):
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}')

print('Finished Training')

Epoch 1, Loss: 0.16728809822488752
Epoch 2, Loss: 0.050070810933356115
Epoch 3, Loss: 0.032514183760787206
Epoch 4, Loss: 0.0244956213066935
Epoch 5, Loss: 0.016720643780073884
Finished Training


# Testing

In [4]:
# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')

Accuracy of the network on the 10000 test images: 99.0%
